In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Define project directories
base_dir = '/content/drive/MyDrive/Datasetr/'
dataset_dir = f'{base_dir}Dataset/'
video_segments_dir = [f'{base_dir}video_segments_{i}' for i in range(1, 12)]
results_dir = f'{base_dir}processed_frames/'


Mounted at /content/drive


**Processed Frames**

In [ ]:
pip install mediapipe opencv-python moviepy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 23.5 MB/s eta 0:00:00


In [ ]:
pip install librosa numpy pydub

In [ ]:
import librosa
import numpy as np
import os
import json
from pydub import AudioSegment

# Define directories
audio_dir = "/content/drive/MyDrive/Datasetr/Dataset/Audio/"
processed_audio_dir = "/content/drive/MyDrive/Datasetr/processed_audio/"
os.makedirs(processed_audio_dir, exist_ok=True)

def process_audio(audio_path, output_path):
    # Convert .mp3 to .wav using pydub
    audio = AudioSegment.from_mp3(audio_path)
    temp_wav_path = audio_path.replace(".mp3", ".wav")
    audio.export(temp_wav_path, format="wav")

    # Load audio file
    y, sr = librosa.load(temp_wav_path, sr=None)

    # Extract features
    features = {
        "chroma_stft": librosa.feature.chroma_stft(y=y, sr=sr).mean(axis=1).tolist(),#captures energy distribution in audio and tonal and harmonic content identify->quality and variation in speech
        "rmse": float(librosa.feature.rms(y=y).mean()),#avg. energy in the audio, loudness variations in audio
        "spectral_centroid": float(librosa.feature.spectral_centroid(y=y, sr=sr).mean()),#COM of spectrum, speech clarity and vocal tone characteristics
        "spectral_bandwidth": float(librosa.feature.spectral_bandwidth(y=y, sr=sr).mean()),#Range of frequencies, DYnamic speech/Monotonous Tone.
        "rolloff": float(librosa.feature.spectral_rolloff(y=y, sr=sr).mean()),#CUtoff frequency< , Speech energy distibution->sharp or smooth transients.
        "zero_crossing_rate": float(librosa.feature.zero_crossing_rate(y).mean()),#Voiced / Unvoiced part of speech difference.
        "mfcc": librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13).mean(axis=1).tolist()#Human ear perception of sound, timbral and phonetic charac.,
    }

    # Save features as JSON
    with open(output_path, "w") as f:
        json.dump(features, f)

    # Remove temporary .wav file
    os.remove(temp_wav_path)

# Process all audio files
for audio_file in os.listdir(audio_dir):
    if audio_file.endswith(".mp3"):
        audio_path = os.path.join(audio_dir, audio_file)
        output_path = os.path.join(processed_audio_dir, f"{os.path.splitext(audio_file)[0]}.json")
        process_audio(audio_path, output_path)



In [ ]:
pip install nltk textblob


In [ ]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import os
import json
import numpy as np
from textblob import TextBlob
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk

# Download NLTK punkt tokenizer
nltk.download("punkt", download_dir='/root/nltk_data')
nltk.data.path.append('/root/nltk_data/')

# Directories
transcripts_dir = "/content/drive/MyDrive/Datasetr/Dataset/Transcripts/"
processed_transcripts_dir = "/content/drive/MyDrive/Datasetr/processed_transcripts/"
os.makedirs(processed_transcripts_dir, exist_ok=True)

# Process transcript using NLTK
def process_transcript_nltk(transcript_path, output_path):
    with open(transcript_path, "r") as f:
        text = f.read()

    # Tokenize words and sentences
    words = word_tokenize(text)
    sentences = sent_tokenize(text)

    #Low sentence counts paired with short sentences could indicate thought blocking or reduced speech fluency.
    #Extreme sentiment polarity or high subjectivity might point to emotional dysregulation or delusional content.

    # Extract features
    features = {
        "word_count": len(words),#total number of words in text
        "sentence_count": len(sentences),#total sentences
        "average_word_length": np.mean([len(word) for word in words]),# Mean of characters per word in text->complexity of language used.
        "average_sentence_length": np.mean([len(sent.split()) for sent in sentences]),#Mean of words per sentence.->Disorganized speech.
        "sentiment_polarity": TextBlob(text).sentiment.polarity,#emtotionsl tone, constant polarity-> depression.
        "sentiment_subjectivity": TextBlob(text).sentiment.subjectivity#Subjective /opionated, Delusional thinking.
    }

    # Save features as JSON
    with open(output_path, "w") as f:
        json.dump(features, f)

# Alternative: Process transcript using spaCy (if NLTK fails)
import spacy
nlp = spacy.load("en_core_web_sm")

def process_transcript_spacy(transcript_path, output_path):
    with open(transcript_path, "r") as f:
        text = f.read()

    doc = nlp(text)
    words = [token.text for token in doc if token.is_alpha]
    sentences = list(doc.sents)

    features = {
        "word_count": len(words),
        "sentence_count": len(sentences),
        "average_word_length": sum(len(word) for word in words) / len(words),
        "average_sentence_length": sum(len(sent) for sent in sentences) / len(sentences),
        "sentiment_polarity": TextBlob(text).sentiment.polarity,
        "sentiment_subjectivity": TextBlob(text).sentiment.subjectivity
    }

    # Save features as JSON
    with open(output_path, "w") as f:
        json.dump(features, f)

# Process all transcripts
for transcript_file in os.listdir(transcripts_dir):
    if transcript_file.endswith(".txt"):
        transcript_path = os.path.join(transcripts_dir, transcript_file)
        output_path = os.path.join(processed_transcripts_dir, f"{os.path.splitext(transcript_file)[0]}.json")
        try:
            process_transcript_nltk(transcript_path, output_path)
        except Exception as e:
            print(f"NLTK failed for {transcript_file}: {e}")
            print(f"Falling back to spaCy for {transcript_file}")
            process_transcript_spacy(transcript_path, output_path)

print("Processing completed. JSON files are saved in:", processed_transcripts_dir)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


NLTK failed for Interview_3.txt: 
**********************************************************************
  Resource punkt_tab not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt_tab')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt_tab/english/

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/root/nltk_data/'
**********************************************************************

Falling back to spaCy for Interview_3.txt
NLTK failed for Interview-4.txt: 
**********************************************************************
  Resource punkt_tab not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt_tab')
  
  For more

In [ ]:
#Video frames processing
import os
import cv2
import mediapipe as mp
import json

# Initialize Mediapipe
mp_face_mesh = mp.solutions.face_mesh
mp_pose = mp.solutions.pose

face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1)
pose = mp_pose.Pose(static_image_mode=True)

# Process individual video frames from a .mp4 file
def process_video(video_path, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    # Capture video
    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_results = face_mesh.process(rgb_frame)
        pose_results = pose.process(rgb_frame)

        # Extract features
        features = {"face_landmarks": [], "pose_landmarks": []}
        if face_results.multi_face_landmarks:
            for landmarks in face_results.multi_face_landmarks:
                features["face_landmarks"] = [
                    {"x": lm.x, "y": lm.y, "z": lm.z} for lm in landmarks.landmark
                ]

        if pose_results.pose_landmarks:
            features["pose_landmarks"] = [
                {"x": lm.x, "y": lm.y, "z": lm.z, "visibility": lm.visibility}
                for lm in pose_results.pose_landmarks.landmark
            ]

        # Save features for each frame
        output_file = os.path.join(output_dir, f"frame_{frame_count:04d}.json")
        with open(output_file, "w") as f:
            json.dump(features, f)

        frame_count += 1

    cap.release()

# Process all video segments
for i, video_segment in enumerate(video_segments_dir, 1):
    video_files = [f for f in os.listdir(video_segment) if f.endswith('.mp4')]
    output_segment_dir = os.path.join(results_dir, f'video_segments_{i}')
    os.makedirs(output_segment_dir, exist_ok=True)

    for video_file in video_files:
        video_path = os.path.join(video_segment, video_file)
        output_dir = os.path.join(output_segment_dir, os.path.splitext(video_file)[0])
        process_video(video_path, output_dir)


KeyboardInterrupt: 

In [ ]:
import os
import cv2
import mediapipe as mp
import json

# Initialize Mediapipe
mp_face_mesh = mp.solutions.face_mesh
mp_pose = mp.solutions.pose

face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1)
pose = mp_pose.Pose(static_image_mode=True)

# Process individual video frames from a .mp4 file
def process_video(video_path, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    # Capture video
    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_results = face_mesh.process(rgb_frame)
        pose_results = pose.process(rgb_frame)

        # Extract features
        features = {"face_landmarks": [], "pose_landmarks": []}
        if face_results.multi_face_landmarks:
            for landmarks in face_results.multi_face_landmarks:
                features["face_landmarks"] = [
                    {"x": lm.x, "y": lm.y, "z": lm.z} for lm in landmarks.landmark
                ]

        if pose_results.pose_landmarks:
            features["pose_landmarks"] = [
                {"x": lm.x, "y": lm.y, "z": lm.z, "visibility": lm.visibility}
                for lm in pose_results.pose_landmarks.landmark
            ]

        # Save features for each frame
        output_file = os.path.join(output_dir, f"frame_{frame_count:04d}.json")
        with open(output_file, "w") as f:
            json.dump(features, f)

        frame_count += 1

    cap.release()

# Directories
video_segments_dir = [
    f"/content/drive/MyDrive/Datasetr/video_segments_{i}" for i in range(1, 12)
]
results_dir = "/content/drive/MyDrive/Datasetr/processed_frames/"
os.makedirs(results_dir, exist_ok=True)

# Process only video segment 10
video_segment = video_segments_dir[8]  # Segment 10 corresponds to index 9 (0-based index)
video_files = [f for f in os.listdir(video_segment) if f.endswith('.mp4')]
output_segment_dir = os.path.join(results_dir, "video_segment_9")
os.makedirs(output_segment_dir, exist_ok=True)

for video_file in video_files:
    video_path = os.path.join(video_segment, video_file)
    output_dir = os.path.join(output_segment_dir, os.path.splitext(video_file)[0])
    process_video(video_path, output_dir)

In [ ]:
import os
import cv2
import json
import mediapipe as mp

# Initialize Mediapipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=5)

def analyze_video_face_postures(video_path, output_json_path):

    # Ensure the video file exists
    if not os.path.exists(video_path):
        print(f"Error: Video file '{video_path}' not found.")
        return

    # Initialize video capture
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Unable to open video '{video_path}'.")
        return

    # Initialize results container
    results = []

    frame_count = 0
    print(f"Processing video: {video_path}...")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to RGB for Mediapipe processing
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_results = face_mesh.process(rgb_frame)

        # Extract face landmarks if detected
        frame_data = {"frame": frame_count, "faces": []}
        if face_results.multi_face_landmarks:
            for face_landmarks in face_results.multi_face_landmarks:
                landmarks = [
                    {"x": lm.x, "y": lm.y, "z": lm.z}
                    for lm in face_landmarks.landmark
                ]
                frame_data["faces"].append(landmarks)

        # Append frame data to results
        results.append(frame_data)

        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames...")

    cap.release()

    # Save results to JSON
    with open(output_json_path, "w") as f:
        json.dump(results, f, indent=4)

    print(f"Processing complete. Results saved to '{output_json_path}'.")

# Process all interview videos
videos_dir = "/content/drive/MyDrive/Datasetr/Dataset/Videos"
output_dir = "/content/drive/MyDrive/Datasetr/processed_face_postures"
os.makedirs(output_dir, exist_ok=True)

# Iterate through all video files in the directory
for video_file in sorted(os.listdir(videos_dir)):
    if video_file.endswith(".mp4"):  # Ensure only .mp4 files are processed
        video_path = os.path.join(videos_dir, video_file)
        output_json_path = os.path.join(output_dir, f"{os.path.splitext(video_file)[0]}_face_postures.json")
        analyze_video_face_postures(video_path, output_json_path)

Processing video: /content/drive/MyDrive/Datasetr/Dataset/Videos/Interview_1.mp4...
Processed 100 frames...
Processed 200 frames...
Processed 300 frames...
Processed 400 frames...
Processed 500 frames...
Processed 600 frames...
Processed 700 frames...
Processed 800 frames...
Processed 900 frames...
Processed 1000 frames...
Processed 1100 frames...
Processed 1200 frames...
Processed 1300 frames...
Processed 1400 frames...
Processed 1500 frames...
Processed 1600 frames...
Processed 1700 frames...
Processed 1800 frames...
Processed 1900 frames...
Processed 2000 frames...
Processed 2100 frames...
Processed 2200 frames...
Processed 2300 frames...
Processed 2400 frames...
Processed 2500 frames...
Processed 2600 frames...
Processed 2700 frames...
Processed 2800 frames...
Processed 2900 frames...
Processed 3000 frames...
Processed 3100 frames...
Processed 3200 frames...
Processed 3300 frames...
Processed 3400 frames...
Processed 3500 frames...
Processed 3600 frames...
Processed 3700 frames...
